In [102]:
import numpy as np
import pandas as pd
import matplotlib

import bokeh
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.palettes import cividis
from bokeh.transform import factor_cmap

In [8]:
artists = pd.read_csv("../../data/LastFM!Music/artists.dat", sep='\t')
artists.head()

,id,name,url,pictureURL
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


In [39]:
tags = pd.read_csv("../../data/LastFM!Music/tags.dat", sep='\t', encoding = 'windows-1251')
#chage to UTF-8
tags['tagID'] = list(map(lambda x: int(str(x).encode('utf-8','strict')), tags['tagID']))
tags['tagValue'] = list(map(lambda x: x.encode('utf-8','strict').decode('utf-8'), tags['tagValue']))
tags.head()

,tagID,tagValue
0,1,metal
1,2,alternative metal
2,3,goth rock
3,4,black metal
4,5,death metal


In [40]:
user_taggedartists = pd.read_csv("../../data/LastFM!Music/user_taggedartists.dat", sep='\t')
user_taggedartists.head()

,userID,artistID,tagID,day,month,year
0,2,52,13,1,4,2009
1,2,52,15,1,4,2009
2,2,52,18,1,4,2009
3,2,52,21,1,4,2009
4,2,52,41,1,4,2009


In [141]:
# there is a probleb
# what are they 1956 / 1957 / 1979
print(user_taggedartists.year.unique())

[2009 2010 2007 2008 2011 2006 2005 1956 1957 1979]


In [144]:
# Ok, just drop it
user_taggedartists.groupby('year')['tagID'].count()

year
1956        3
1957        1
1979        1
2005     2483
2006     9814
2007    20415
2008    40273
2009    43366
2010    54998
2011    15125
Name: tagID, dtype: int64

In [49]:
user_artists = pd.read_csv("../../data/LastFM!Music/user_artists.dat", sep='\t')
user_artists.head()

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [126]:
user_artists_tags = user_taggedartists.merge(user_artists,
                                       how = 'left',
                                       left_on = ['userID', 'artistID'],
                                       right_on = ['userID', 'artistID'])
user_artists_tags = user_artists_tags.merge(tags,
                                       how = 'left',
                                       left_on = 'tagID',
                                       right_on = 'tagID')[['userID', 'artistID', 'tagID', 'tagValue']]
user_artists_tags.head()

,userID,artistID,tagID,tagValue
0,2,52,13,chillout
1,2,52,15,downtempo
2,2,52,18,electronic
3,2,52,21,trip-hop
4,2,52,41,female vovalists


In [137]:
grp = user_artists_tags.groupby('tagValue')['userID'].count()

#select top 50 tags
tags = grp.nlargest(50).index.tolist()
counts = grp.nlargest(50).values.tolist()

#create bokeh polot
p = figure(x_range=tags, 
           plot_height=350, 
           title="Top 50 out of {} tags counts".format(len(grp.index.tolist())),
           toolbar_location=None, 
           tools="")

p.vbar(x=tags, top=counts, width=0.9)
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = 3.14/2

show(p)

In [139]:
user_taggedartists.year.unique()

array([2009, 2010, 2007, 2008, 2011, 2006, 2005, 1956, 1957, 1979],
      dtype=int64)